In [394]:
import sqlite3
from Bio import SeqIO

class CodonDatabase:
    def __init__(self,database=None):
        if database != None:
            self.conn = sqlite3.connect(database)
    
    # Database schema of the Codon Database
    CREATE_DB ="""
        CREATE TABLE IF NOT EXISTS 'amino_acids' (
            'amino_acid' TEXT PRIMARY KEY
        );

        CREATE TABLE IF NOT EXISTS 'organisms' (
            'id' TEXT PRIMARY KEY,
            'description' TEXT NOT NULL,
            'organism' TEXT NOT NULL
        );

        CREATE TABLE IF NOT EXISTS 'transl_tables' (
            'transl_table' INT PRIMARY KEY
        );

        CREATE TABLE IF NOT EXISTS 'codons' (
            'codon' TEXT NOT NULL,
            'codon_count' INTEGER NOT NULL DEFAULT 0,
            'amino_acid' TEXT NOT NULL,
            'transl_table' INTEGER NOT NULL,
            'organism' TEXT NOT NULL,
            FOREIGN KEY('amino_acid') REFERENCES 'amino_acids'('amino_acid'),
            FOREIGN KEY('transl_table') REFERENCES 'transl_tables'('transl_table'),
            FOREIGN KEY('organism') REFERENCES 'organisms'('id')
            PRIMARY KEY('codon', 'amino_acid', 'organism')
        );

        """
        # transl_tables are for 3NF standardization
    
    def choose_codon(self,organism_id,aa,banned=[]):
        # Check that the amino acid is valid
        valid_aa = 'ARNDBCEQZGHILKMFPSTWYV*'
        if aa not in valid_aa:
            return ValueError('{} not valid amino acid ({})'.format(aa,valid_aa))
        # 1. Build a table with a certain amino acid and a certain codon without any of the banned codons
        # 2. Take cumulative probability and, using a random number input from python, select a codon
        random_select = """
            WITH codons_selective(codon,amino_acid,codon_count) AS (
                SELECT codon, amino_acid, codon_count FROM codons WHERE amino_acid = ? AND organism = ? AND codon NOT IN {}
                )
            SELECT codon, CAST((SELECT sum(codon_count) FROM codons_selective AS c WHERE c.codon <= codons_selective.codon) AS FLOAT) / (SELECT sum(codon_count) FROM codons_selective) as probability FROM codons_selective
            WHERE probability >= ?
            ORDER BY probability ASC
            LIMIT 1;
            """ # https://stackoverflow.com/questions/50534961/sqlite-how-to-select-rows-based-on-probability-via-integer-value
        try:
            # Select a codon. First, add in a variable number of ? for banned codons, then run above statement. Fetch first codon of first result
            r = self.conn.cursor().execute(random_select.format('(' + ','.join(["?" for _ in banned]) + ')'), (aa,organism_id,)+tuple(banned)+(random.uniform(0,1),)).fetchone()[0]
        except Exception as e:
            # If there is a value error, it is likely that the organism is not in the database
            raise ValueError('Organism or amino acid not found in database')
        return r
    
    def optimize_sequence(self,organism_id,protein_seq):
        return ''.join([self.choose_codon(organism_id, codon) for codon in protein_seq])
    
    def build_from_genbank(self,genbank_file):
        self.conn = sqlite3.connect(':memory:')
        c = self.conn.cursor()
        
        for x in self.CREATE_DB.split(';'):
            c.execute(x)

        # Add amino acids
        amino_acids = 'ARNDBCEQZGHILKMFPSTWYV*'
        c.executemany('INSERT OR IGNORE INTO amino_acids(amino_acid) VALUES(?)', [(x,) for x in amino_acids])

        #Add codons
        from Bio import SeqIO
        for record in SeqIO.parse(genbank_file, "genbank"):
            c.execute('INSERT OR IGNORE INTO organisms(id,description,organism) VALUES(?,?,?)', (record.id, record.description,record.annotations['organism']))
            for feature in record.features:
                if 'translation' in feature.qualifiers:
                    translation = feature.qualifiers['translation'][0] + '*'
                    seq = feature.extract(record).seq
                    c.execute('INSERT OR IGNORE INTO transl_tables(transl_table) VALUES(?)', (int(feature.qualifiers['transl_table'][0]),))
                    c.executemany('INSERT OR IGNORE INTO codons(amino_acid,codon,organism,transl_table) VALUES(?,?,?,?)', [(aa,str(seq[i*3:i*3+3]),record.id,int(feature.qualifiers['transl_table'][0])) for i,aa in enumerate(translation)])
                    c.executemany('UPDATE codons SET codon_count = codon_count + 1 WHERE amino_acid = ? AND codon = ? AND organism = ?', [(aa,str(seq[i*3:i*3+3]),record.id) for i,aa in enumerate(translation)])

        self.conn.commit()
        return self
    
    def dump_codon_database(self):
        return '\n'.join([line for line in self.conn.iterdump()])


In [395]:
c = CodonDatabase().build_from_genbank('sequence.gb')

In [396]:
c.dump_codon_database()

'BEGIN TRANSACTION;\nCREATE TABLE \'amino_acids\' (\n            \'amino_acid\' TEXT PRIMARY KEY\n        );\nINSERT INTO "amino_acids" VALUES(\'A\');\nINSERT INTO "amino_acids" VALUES(\'R\');\nINSERT INTO "amino_acids" VALUES(\'N\');\nINSERT INTO "amino_acids" VALUES(\'D\');\nINSERT INTO "amino_acids" VALUES(\'B\');\nINSERT INTO "amino_acids" VALUES(\'C\');\nINSERT INTO "amino_acids" VALUES(\'E\');\nINSERT INTO "amino_acids" VALUES(\'Q\');\nINSERT INTO "amino_acids" VALUES(\'Z\');\nINSERT INTO "amino_acids" VALUES(\'G\');\nINSERT INTO "amino_acids" VALUES(\'H\');\nINSERT INTO "amino_acids" VALUES(\'I\');\nINSERT INTO "amino_acids" VALUES(\'L\');\nINSERT INTO "amino_acids" VALUES(\'K\');\nINSERT INTO "amino_acids" VALUES(\'M\');\nINSERT INTO "amino_acids" VALUES(\'F\');\nINSERT INTO "amino_acids" VALUES(\'P\');\nINSERT INTO "amino_acids" VALUES(\'S\');\nINSERT INTO "amino_acids" VALUES(\'T\');\nINSERT INTO "amino_acids" VALUES(\'W\');\nINSERT INTO "amino_acids" VALUES(\'Y\');\nINSERT I